# LZ78 [[Ziv and Lempel, 1978]](https://scholar.google.es/scholar?hl=es&as_sdt=0%2C5&q=Ziv+Lempel+1978&btnG=)

* In 1978, Ziv and Lempel published the [LZ78 algorithm](https://en.wikipedia.org/wiki/LZ77_and_LZ78).

* LZ78 represents the dictionary in a recursive way with the idea
  of reducing the memory used for representing the strings in the dictionary. Now, each
  entry in the dictionary is a pair $wk$, where $w$ is an index to
  an entry of the dictionary and $k$ is a symbol. In fact, each pair $wk$
  represents the string that results from the concatenation of the string
  $w$ and the symbol $k$, where $w$ can be recursively computed in the same way
  we have found $wk$.
  
* We will denote the string that $w$ represents by *string*$(w)$.
  
* The empty string is obtained by *string*$(0)$.

## Encoder

1. $w\leftarrow 0$.
2. While the input is not exhausted:
    1. $k\leftarrow$ next input symbol.
    2. If $wk$ is found in the dictionary, then:
        1. $w\leftarrow$ address of $wk$ in the dictionary.
    3. Else:
        1. Output $wk$.
        2. Insert $wk$ in the dictionary.
        3. $w\leftarrow 0$.

## Example
<img src="data/LZ78_encoding_example.png" width="700">

## Decoder

1. While the input is not exhausted:
    1. Input $wk$.
    2. Output $\text{string}(w)$.
    3. Output $k$.
    4. Insert $wk$ in the dictionary.

## Example
<img src="data/LZ78_decoding_example.png" width="600">

## Lab

In [2]:
import io

def encoder(uncompressed):
    """Compress a string to a list of output symbols."""

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}

    w = ""
    result = []
    for k in uncompressed:
        wk = w + k
        if wk in dictionary:
            w = wk
        else:
            result.append(dictionary[w])
            # Add wk to the dictionary.
            dictionary[wk] = dict_size
            dict_size += 1
            w = k

    # Output the code for w.
    if w:
        result.append(dictionary[w])
    return result


def decoder(compressed):
    """Decompress a list of output ks to a string."""

    # Build the dictionary.
    dict_size = 256
    dictionary = {chr(i): chr(i) for i in range(dict_size)}

    # use io.StringIO(), otherwise this becomes O(N^2)
    # due to string concatenation in a loop
    result = io.StringIO()
    w = compressed.pop(0)
    result.write(w)
    for k in compressed:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            entry = w + w[0]
        else:
            raise ValueError('Bad compressed k: %s' % k)
        result.write(entry)

        # Add w+entry[0] to the dictionary.
        dictionary[dict_size] = w + entry[0]
        dict_size += 1

        w = entry
    return result.getvalue()


# Testing the algorithm
cadena = "Hola, soy Claudio y este algoritmo es correcto..."
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Chains in which we can see this algorithm compresses much more
cadena = "AABBAAABBBAAAABBBBAAAAABBBBBABABA"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

print()

# Another example
cadena = "TOBEORNOTTOBEORTOBEORNOT"
compressed = encoder(cadena)
print (compressed)
decompressed = decoder(compressed)
print (decompressed)

['H', 'o', 'l', 'a', ',', ' ', 's', 'o', 'y', ' ', 'C', 258, 'u', 'd', 'i', 'o', ' ', 264, 'e', 's', 't', 'e', ' ', 'a', 'l', 'g', 'o', 'r', 'i', 't', 'm', 271, 274, ' ', 'c', 282, 'r', 'e', 'c', 't', 'o', '.', 297]
Hola, soy Claudio y este algoritmo es correcto...

['A', 'A', 'B', 'B', 256, 257, 258, 260, 261, 262, 263, 258, 267, 257, 269]
AABBAAABBBAAAABBBBAAAAABBBBBABABA

['T', 'O', 'B', 'E', 'O', 'R', 'N', 'O', 'T', 256, 258, 260, 265, 259, 261, 263]
TOBEORNOTTOBEORTOBEORNOT
